# Stage 4: Creating neural networks

前面我们主要处理的变量是标量，但在机器学习领域，张量(多维数组)扮演着重要的角色。  
这一阶段的目标是将 DeZero 扩展到机器学习，尤其是神经网络领域。

In [1]:
import numpy as np
import dezero.functions as F
from dezero import Variable

## Step 37: Processing tensors

前面，我们处理的变量主要是标量。本步骤将将讨论使用张量时需要注意的地方，并为扩展DeZero做准备。

In [3]:
x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

variable(0.8414709848078965)


In [5]:
# 会对多维数组进行逐元素运算，不改变形状
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sin(x)
print(y)
# 逐元素相加
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
c = Variable(np.array([[10, 20, 30], [40, 50, 60]]))
y = x + c
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])
variable([[11 22 33]
          [44 55 66]])


在使用当前己实现函数的情况下，即使对张量进行计算，反向传播的代码也能
正常工作：
1. 以标量为对象实现了反向传播
2. 向目前实现的 DeZero 函数传入张量，函数会将每个张量的元素作为标量进行计算
3. 如果将张量的每个元素作为标量进行计算，那么以标量为前提实现的
反向传播也会对张量的每个元素进行计算

$\mathbf{y} = F(\mathbf{x})$向量求导（雅可比矩阵）：
$$
\begin{align*}
\frac{\partial \mathbf{y}}{\partial \mathbf{x}} = \left(
\begin{array}{ccc}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \cdots & \frac{\partial y_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \frac{\partial y_m}{\partial x_2} & \cdots & \frac{\partial y_m}{\partial x_n}
\end{array}
\right)
\end{align*}
$$
如果$\mathbf{y}$是标量，那么就只有一行$\left(\frac{\partial y}{\partial x_1}, \frac{\partial y}{\partial x_2}, \cdots, \frac{\partial y}{\partial x_n}\right)$。

接下来考虑复合函数，基于链式法则，可以得到：
$$
\begin{align*}
\frac{\partial \mathbf{y}}{\partial \mathbf{x}} = \frac{\partial \mathbf{y}}{\partial \mathbf{a}} \frac{\partial \mathbf{a}}{\partial \mathbf{b}} \frac{\partial \mathbf{b}}{\partial \mathbf{x}}
\end{align*}
$$
计算通过矩阵乘法实现。


## Step 38: Function for changing shape

现在要看不逐元素进行运算的函数-reshape、transpose等。

reshape函数只是对形状进行变换，不进行具体的计算。因此在反向传播时，只需将梯度转换为原始形状传递给上游即可。


In [2]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad=True)
print(y.grad)
print(x.grad)

variable([1 1 1 1 1 1])
variable([[1 1 1]
          [1 1 1]])


numpy的ndarray实例可以直接使用reshape方法，为了让函数更易用，在Variable类中实现reshape方法。

In [2]:
x = Variable(np.random.randn(1,2,3))
print(x)
y = x.reshape((2,3))
print(y)
y = x.reshape(2,3)
print(y)

variable([[[-1.25690037  0.1060291   0.75893363]
           [-0.64814541  0.06557808  1.41735456]]])
variable([[-1.25690037  0.1060291   0.75893363]
          [-0.64814541  0.06557808  1.41735456]])
variable([[-1.25690037  0.1060291   0.75893363]
          [-0.64814541  0.06557808  1.41735456]])


transpose转置函数与reshape函数实现方式类似，只需将形状变换即可。

In [3]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = x.transpose()
print(y)
y = x.T
print(y)
y.backward(retain_grad=True)
print(y.grad)
print(x.grad)

variable([[1 4]
          [2 5]
          [3 6]])
variable([[1 4]
          [2 5]
          [3 6]])
variable([[1 1]
          [1 1]
          [1 1]])
variable([[1 1 1]
          [1 1 1]])


## Step 39: Funciton for summing elements

之前实现的加法函数反向传播时，将梯度直接传递给两个输入变量。

现在实现的函数对张量的元素进行求和也同理，反向传播时，将梯度传递给所有元素，即按向量元素数量复制梯度。

而复制操作也是操作Variable类的方法，需要实现broadcast_to方法。

另外np的sum函数可以指定axis参数，实现对指定轴求和，这里也要实现这个功能。

In [3]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

x.cleargrad()
y = x.sum(axis=0)
y.backward()
print(y)
print(x.grad)

x = Variable(np.random.randn(2,3,4,5))
y = x.sum(keepdims=True)
print(y.shape)

variable(21)
variable([[1 1 1]
          [1 1 1]])
variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])
(1, 1, 1, 1)


## Step 40: Function for broadcasting

y = x + x可以理解为复制x后再相加，在反向传播时，需要将梯度累加到x上。

广播操作也是类似原理，在反向传播时，需要将梯度累加到原始形状上。

这里需要使用sum_to函数，进行求和并将形状变换回去。

而sum_to函数的反向传播实现是将梯度复制到原始形状上，则使用broadcast_to函数。

另外，np中的广播操作往往是幕后进行的，所以需要拓展原来的四则运算函数，使其支持广播操作。

In [2]:
x0 = Variable(np.array([1, 2, 3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
print(x1)
print(x1.grad)

variable([11 12 13])
variable([10])
variable([3])


## Step 41: Product of matrices

向量内积：
$$
\begin{align*}
\mathbf{y} = \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i
\end{align*}
$$
矩阵乘法：
$$
\begin{align*}
\mathbf{Y} = \mathbf{A} \mathbf{B} = \left(
\begin{array}{ccc}
\sum_{i=1}^{n} a_{1i} b_{i1} & \sum_{i=1}^{n} a_{1i} b_{i2} & \cdots & \sum_{i=1}^{n} a_{1i} b_{im} \\
\sum_{i=1}^{n} a_{2i} b_{i1} & \sum_{i=1}^{n} a_{2i} b_{i2} & \cdots & \sum_{i=1}^{n} a_{2i} b_{im} \\
\vdots & \vdots & \ddots & \vdots \\
\sum_{i=1}^{n} a_{mi} b_{i1} & \sum_{i=1}^{n} a_{mi} b_{i2} & \cdots & \sum_{i=1}^{n} a_{mi} b_{im}
\end{array}
\right)
\end{align*}
$$

都使用np.dot函数实现，如果两个参数都是一维数组，则是向量内积；如果两个参数都是二维数组，则是矩阵乘法。

<center>
<table>
  <tr>
    <td><img src="./res/matmul_easy.png" width="400"/></td>
  </tr>
</table>
</center>

对于反向传播，首先考虑简单情况 $y = xW$，其中x为1×D的向量，W为D×H的矩阵，y为1×H的向量。
$$
\begin{align*}
\frac{\partial L}{\partial x_i} = \sum_{j=1}^{H} \frac{\partial L}{\partial y_j} \frac{\partial y_j}{\partial x_i} = \sum_{j=1}^{H} \frac{\partial L}{\partial y_j} W_{ij}
\end{align*}
$$
其中，$W_{ij}$表示W的第i行第j列元素。

当$x_i$发生变化时，向量$y$的所有元素都会发生变化。$y$的每个元素改变也会使$L$发生变化，因此从$x_i$到$L$有多条链式法则的路径。

由前可知，$\frac{\partial L}{\partial x_i}$可通过向量$\frac{\partial L}{\partial y}$和矩阵$W$的第i行向量的内积得到，因此可以推广得到：
$$
\begin{align*}
\frac{\partial L}{\partial \mathbf{x}} = \frac{\partial L}{\partial \mathbf{y}} \mathbf{W}^T
\end{align*}
$$

<center>
<table>
  <tr>
    <td><img src="./res/matmul_complex.png" width="400"/></td>
    <td><img src="./res/matmul_grad.png" width="350"/></td>
  </tr>
</table>
</center>

In [2]:
x = Variable(np.random.randn(2,3))
W = Variable(np.random.randn(3,4))
y = F.matmul(x, W)
y.backward()

print(x)
print(x.grad.shape)
print(W)
print(W.grad.shape)

variable([[-0.2218563  -1.86013392  0.04561382]
          [ 0.44896001  1.74007878  0.41856793]])
(2, 3)
variable([[-1.74508997  0.53697346  0.04406799  0.73966956]
          [ 0.88525435 -1.52196217  0.1616612   0.8558494 ]
          [ 1.04271278  0.30658378 -1.54890008  0.21894359]])
(3, 4)
